In [ ]:
# from datasets import load_dataset
# from training_datasets.GLUEDataset import GLUEDataset

# cola = load_dataset('glue','cola')
# # print(cola)
# # sst2 = load_dataset('glue','sst2')
# # print(sst2)
# mrpc = load_dataset('glue', 'mrpc')
# # print(mrpc)
# pt_cola = GLUEDataset([cola, mrpc], 'train')

In [ ]:
from training_datasets.GLUEMetaDataset import GLUEMetaDataset
from samplers.FewShotEpisodeSampler import FewShotEpisodeSampler
from samplers.FewShotEpisodeBatchSampler import FewShotEpisodeBatchSampler
import torch.utils.data as data

meta_ds = GLUEMetaDataset(k=4,numTasks=1000)

In [ ]:
train_protomaml_sampler = FewShotEpisodeBatchSampler(meta_ds, kShot=4, batchSize=4)

train_protomaml_loader = data.DataLoader(
    meta_ds, batch_sampler=train_protomaml_sampler, collate_fn=train_protomaml_sampler.getCollateFunction(), num_workers=4)

# for i in range(1):
#     batch = next(iter(train_protomaml_loader))
#     for episode_i in range(len(batch[0])):
#         data, labels = batch[0][episode_i], batch[1][episode_i]
#         supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)

In [ ]:
# episodes = []
# for i in range(1):
#     episodes.append(gm_ds.getTask())
# for i in range(len(episodes)):
#     classes = len(set(episodes[i][1].tolist()))
#     # print(classes)
#     print(episodes[i][0], episodes[i][1], "\n")

In [ ]:
# from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.training_step(next(iter(train_protomaml_loader)), 0)

In [4]:
from samplers.FewShotValidationEpisodeBatchSampler import FewShotValidationEpisodeBatchSampler
from samplers.FewShotValidationEpisodeSampler import FewShotValidationEpisodeSampler
from validation_datasets.ValidationDataset import ValidationDataset
import torch.utils.data as data

ds = ValidationDataset()

val_protomaml_sampler = FewShotValidationEpisodeBatchSampler(ds, kShot=4)
val_protomaml_loader = data.DataLoader(
    ds, batch_sampler=val_protomaml_sampler, collate_fn=val_protomaml_sampler.getCollateFunction(), num_workers=1
)

for i in range(1):
    batch = next(iter(val_protomaml_loader))
    for episode_i in range(len(batch[0])):
        data, labels = batch[0][episode_i], batch[1][episode_i]
        supportSet, supportLabels = data[0:len(data)//2], labels[0:len(data)//2] 
#         print(supportLabels)
        print(labels)

[1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0]
[3, 4, 2, 4, 3, 2, 3, 4, 2, 3, 2, 4, 2, 4, 4, 2, 3, 3, 4, 3, 2, 3, 4, 2]
[6, 7, 5, 5, 6, 6, 5, 7, 5, 7, 7, 6, 6, 7, 6, 7, 6, 5, 7, 5, 5, 6, 5, 7]
[10, 10, 9, 8, 9, 10, 9, 8, 8, 8, 9, 10, 10, 8, 10, 9, 10, 9, 9, 9, 8, 10, 8, 8]
[13, 11, 12, 11, 12, 13, 13, 11, 13, 12, 11, 12, 12, 11, 12, 13, 11, 12, 13, 13, 11, 12, 11, 13]
[14, 14, 15, 14, 15, 15, 14, 15, 14, 15, 15, 14, 14, 14, 15, 15]
[16, 17, 17, 17, 16, 17, 16, 16, 16, 17, 17, 17, 16, 16, 16, 17]
[19, 18, 19, 18, 18, 19, 19, 18, 19, 18, 18, 19, 19, 19, 18, 18]
[1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1]
[4, 4, 2, 2, 3, 2, 3, 2, 3, 3, 4, 4, 2, 4, 3, 2, 4, 2, 3, 3, 2, 4, 4, 3]
[5, 6, 7, 5, 7, 5, 7, 6, 7, 5, 6, 6, 6, 5, 5, 7, 7, 5, 6, 6, 7, 6, 5, 7]
[10, 8, 8, 9, 9, 9, 10, 8, 9, 8, 10, 10, 9, 10, 10, 8, 8, 10, 9, 8, 10, 9, 8, 9]
[12, 13, 13, 11, 11, 12, 13, 11, 12, 11, 13, 12, 13, 12, 11, 12, 13, 13, 11, 12, 11, 13, 12, 11]
[14, 15, 14, 14, 15, 14, 15, 15, 15, 14, 15, 14, 15, 14, 14

In [1]:
from training.models.ProtoFOMAML import ProtoFOMAML

# pfomaml = ProtoFOMAML(outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=16, warmupSteps=0)
# pfomaml.validation_step(next(iter(val_protomaml_loader)), 0)

/home/aksingh/miniconda3/envs/meta-learned-lines/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from training.trainer import train_model

protomaml_model = train_model(
    ProtoFOMAML,
    train_loader=train_protomaml_loader,
    val_loader=val_protomaml_loader,
    outerLR=5e-4, innerLR=1e-3, outputLR=1e-2, steps=5, batchSize=4, warmupSteps=0
)

In [3]:
import os

os.path.abspath(os.getcwd())

'/home/aksingh/Documents/meta-learned-lines'